# Illustrating of generation of dataset for `hemibrain` to `flywire` transformation 

## Step 1: Import neccesary library modules now

In [1]:
import navis
import fafbseg
import flybrains

In [2]:
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.flywire import flywireurl2dict, add_flywirelayer, add_flywirehostedlayer
from pyroglancer.skeletons import to_ngskeletons, to_precomputedskels, to_precomputedskelsinfo
from pyroglancer.layers import flush_precomputed, create_nglayer, handle_skels, get_ngserver, add_hostedlayer
from pyroglancer.ngspaces import create_ngspace
from pyroglancer.volumes import to_ngmesh, to_precomputedsingleresmeshes, to_precomputedsingleresmeshesinfo

In [3]:
import navis.interfaces.neuprint as neu
from navis.interfaces.neuprint import NeuronCriteria as NC, SynapseCriteria as SC

In [4]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from time import sleep

In [5]:
navis.set_loggers('ERROR')
navis.set_pbars(hide=True)

In [6]:
def try_until(func, max_tries, sleep_time):
    for _ in range(0,max_tries):
        try:
            return func()
        except:
            sleep(sleep_time)
    raise ValueError('skipping as we dont succeed after max tries..') 

## Step 2: Set configurations to fetch from data from `neuprint`

In [7]:
client = neu.Client('https://neuprint.janelia.org/', dataset='hemibrain:v1.2')

In [8]:
client

Client("https://neuprint.janelia.org", "hemibrain:v1.2")

## Step 3: Get bodyids from the csv file..

In [9]:
csvloc = '/Users/sri/Documents/dataset/EM_Hemibrain_Neuronbridge.csv'

In [10]:
dataset_df = pd.read_csv(csvloc)

In [11]:
status = dataset_df.Line.str.split('_18U', n = 1, expand=True)
status.columns = ['name', 'flip']
res = status.name.str.rsplit('_', 1, expand = True)
res.columns = ['name', 'status']
print('Different status are: ', np.unique(res.status))

Different status are:  ['LV' 'RT' 'TC']


In [12]:
# get only traced and roughly traced..
roughlytraced = res.status.str.contains('RT')
traced = res.status.str.contains('TC')
lvstatus = res.status.str.contains('LV')
selecteddataset_df = res[roughlytraced | traced]
print('Used status are: ', np.unique(selecteddataset_df.status))

Used status are:  ['RT' 'TC']


In [13]:
selectedneurons = selecteddataset_df.name.str.split('-', n = 1, expand=True)
selectedneurons.columns = ['bodyid', 'name']
selectedneurons = selectedneurons.drop_duplicates()
#selectedneurons['bodyid'] = selectedneurons['bodyid'].apply(pd.to_numeric, errors='coerce')
selectedneurons = selectedneurons.reset_index(drop = True)
print('Choosen neurons are: ', len(selectedneurons.bodyid.unique()))

Choosen neurons are:  21666


In [14]:
selectedneurons

,bodyid,name
0,989228019,FB5V
1,988909130,FB5V
2,988852391,PAM08_a
3,988632865,VES045
4,988567837,FB4G
...,...,...
21661,1002507170,PFL1
21662,1002507159,EPG
21663,1002507131,PEN_a(PEN1)
21664,1002360103,AVLP464


## Step 4: Select bodyids that will be fetched, and set host directory..

In [15]:
numbodyids = 5000

In [16]:
bodyids = selectedneurons.bodyid[0:numbodyids]
bodyids = list(map(int, bodyids))

In [17]:
print('Total neurons that will be processed: ', len(bodyids))

Total neurons that will be processed:  5000


In [18]:
hostdirectory = '/Users/sri/Documents/dataset/hemibrain2flywire/'

## Step 5: Get stuff from `neuprint`, transform, precompute them..

In [19]:
flush_precomputed(hostdirectory, '')

deleting.. /Users/sri/Documents/dataset/hemibrain2flywire/precomputed/


## skeleton part..

In [20]:
flush_precomputed(hostdirectory, 'skeletons')

In [21]:
nskelbatch = 25 #number of skeletons to process in one-go..

In [22]:
allskelseglist = []
allskelsegnamelist = []

In [23]:
def transform2flywireskeletons(bodyids):
    neurons_df = neu.fetch_skeletons(bodyids, heal=1000, max_threads=nskelbatch, missing_swc='raise')
    hemibrain_neurons=navis.xform_brain(neurons_df,source='JRCFIB2018Fraw', target='FLYWIRE',verbose=False)
    #hemibrain_neurons=navis.xform_brain(neurons_df,source='JRCFIB2018Fraw', target='FAFB',verbose=False)
    skelsource, skelseglist, skelsegnamelist = to_ngskeletons(hemibrain_neurons)
    to_precomputedskels(skelsource, hostdirectory)
    return skelseglist, skelsegnamelist 

In [24]:
splitlist = np.array_split(bodyids, len(bodyids)/nskelbatch)
#len(np.concatenate(splitlist).ravel())

In [25]:
skippedchunkids = []
chunkidx = 0

In [26]:
for chunkbodyids in tqdm(splitlist):
    #print('processing: ', chunkbodyids)
    #skelseglist, skelsegnamelist = transform2flywireskeletons(chunkbodyids)
    skelseglist, skelsegnamelist = try_until(lambda : transform2flywireskeletons(chunkbodyids), 100, 5)
    allskelseglist += skelseglist
    allskelsegnamelist += skelsegnamelist

/Users/sri/opt/anaconda3/lib/python3.8/site-packages/fafbseg/spine/base.py:303: UserWarning:

1 points failed to transform.

/Users/sri/opt/anaconda3/lib/python3.8/site-packages/fafbseg/spine/base.py:303: UserWarning:

1 points failed to transform.

/Users/sri/opt/anaconda3/lib/python3.8/site-packages/fafbseg/spine/base.py:303: UserWarning:

1 points failed to transform.



In [27]:
print('Processed neuron skeletons are: ', len(allskelseglist))

Processed neuron skeletons are:  5000


In [28]:
to_precomputedskelsinfo(allskelseglist, allskelsegnamelist, hostdirectory)

creating: /Users/sri/Documents/dataset/hemibrain2flywire/precomputed/skeletons/seg_props


## neuronmesh part..

In [29]:
flush_precomputed(hostdirectory, 'neuronmeshes')

In [30]:
neuronmeshbatch = 10 #number of neuron meshes to process in one-go..

In [31]:
allvolumeidlist = []
allvolumenamelist = []

In [32]:
def transform2flywiremeshes(bodyids):
    neuronmeshes_df = neu.fetch_mesh_neuron(bodyids, lod=2, with_synapses=False)
    hemibrainneuronmeshes_df = navis.xform_brain(neuronmeshes_df,source='JRCFIB2018Fraw', target='FLYWIRE',verbose=False)
    #hemibrainneuronmeshes_df = navis.xform_brain(neuronmeshes_df,source='JRCFIB2018Fraw', target='FAFB',verbose=False)
    volumedatasource, volumeidlist, volumenamelist = to_ngmesh(hemibrainneuronmeshes_df)
    to_precomputedsingleresmeshes(volumedatasource, hostdirectory, 'neuronmeshes')
    return volumeidlist, volumenamelist 

In [33]:
splitlist = np.array_split(bodyids, len(bodyids)/neuronmeshbatch)
#len(np.concatenate(splitlist).ravel())

In [34]:
for chunkbodyids in tqdm(splitlist):
    #print('processing: ', chunkbodyids)
    #volumeidlist, volumenamelist = transform2flywiremeshes(chunkbodyids)
    volumeidlist, volumenamelist = try_until(lambda : transform2flywiremeshes(chunkbodyids), 100, 5)
    allvolumeidlist += volumeidlist
    allvolumenamelist += volumenamelist

/Users/sri/opt/anaconda3/lib/python3.8/site-packages/fafbseg/spine/base.py:303: UserWarning:

1 points failed to transform.



In [35]:
print('Processed neuron meshes are: ', len(allvolumeidlist))

Processed neuron meshes are:  5000


In [36]:
to_precomputedsingleresmeshesinfo(allvolumeidlist, allvolumenamelist, hostdirectory, 'neuronmeshes')

creating: /Users/sri/Documents/dataset/hemibrain2flywire/precomputed/neuronmeshes/mesh/segment_properties
creating: /Users/sri/Documents/dataset/hemibrain2flywire/precomputed/neuronmeshes/mesh/segment_names
